In [21]:
## Catalog search 

import numpy as np
import geopandas as gpd
import rioxarray as rioxr
import matplotlib.pyplot as plt

from shapely.geometry import Polygon

# used to access STAC catalogs
from pystac_client import Client
# used to sign items from the MPC STAC catalog
import planetary_computer

# ----- other libraries for nice ouputs
from IPython.display import Image




In [22]:
catalog

<Client id=microsoft-pc>

In [23]:
catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1",
                      modifier = planetary_computer.sign_inplace)
#temporal range of interest
time_range = '2017/2020'
#Bounding Box Coordinates (as a GeoJSON)
bbox = [-112.826843, 32.974108, -111.184387, 33.863574]
#Catalog search
search = catalog.search(
    collections = ['io-biodiversity'],
    bbox = bbox,
    datetime = time_range
)
items = search.item_collection()
items
#Number of items in the search:
len(items)

4